In [1]:
#------------------------------HEADER------------------------------#
#Projeto computacional em que implementamos o método simplex.
'''AUTHORS:
Daniel Reis, Matheus Alves, Mariane Santana.
'''

import numpy as np
#------------------------------HEADER------------------------------#

In [218]:
linhas = int(input("Informe o tamanho da matriz.\nLinhas: "))
colunas = int(input("\nColunas: "))
# print("i, j = ({:1f}, {:1f})\n".format(linhas, colunas))

A = np.zeros(linhas*colunas, dtype=float).reshape(linhas, colunas)

for i in range(linhas):
    A[i] = input().split()

print(A)

[[2. 1. 1. 0. 0.]
 [2. 3. 0. 1. 0.]
 [3. 1. 0. 0. 1.]]


In [219]:
# c = input("Informe o vetor dos custos (separados por um espaço): ").split(sep = " ")
custos = [float(c) for c in input("Informe o vetor dos custos (separados por um espaço): ").split(sep = " ")]

print(custos)

[-3.0, -2.0, 0.0, 0.0, 0.0]


In [220]:
recursos = [float(b) for b in input("Informe o vetor dos recursos (separados por um espaço): ").split(sep = " ")]

print(recursos)

[18.0, 42.0, 24.0]


In [221]:
print(A)
print(custos)
print(recursos)

[[2. 1. 1. 0. 0.]
 [2. 3. 0. 1. 0.]
 [3. 1. 0. 0. 1.]]
[-3.0, -2.0, 0.0, 0.0, 0.0]
[18.0, 42.0, 24.0]


In [222]:
# Define partições e os custos básicos e não-básicos
particao_basica = A[:,:linhas]
particao_nbasica = A[:,(linhas-colunas):]
custos_basicos = custos[:linhas]
custos_nbasicos = custos[(linhas-colunas):]

In [223]:
# Calcula a solução básica factível Xb
xb = np.linalg.solve(particao_basica, recursos)
print(xb)

[ 4.28571429 11.14285714 -1.71428571]


In [224]:
valor_atual = np.dot(np.transpose(custos_basicos), xb)
print(valor_atual)

-35.14285714285714


In [227]:
# Calcular os custos relativos
lambda_simplex = np.linalg.solve(np.transpose(particao_basica), custos_basicos)
custos_relativos = [(custos_nbasicos[j] - np.dot(np.transpose(lambda_simplex), particao_nbasica[:,j])) for j in range(colunas-linhas)]
custo_relativo_min = min(custos_relativos)
indice_entrada_base = custos_relativos.index(custo_relativo_min)

In [228]:
print(lambda_simplex)
print(custos_relativos)
print(custo_relativo_min)
print(indice_entrada_base)

[ 0.         -0.42857143 -0.71428571]
[0.4285714285714286, 0.7142857142857143]
0.4285714285714286
0


In [229]:
# Teste de otimalidade
if (custo_relativo_min >= 0):
    print("Solução atual é ótima.")
    print("Valor ótimo da solução: ", valor_atual)
    print("Variáveis ótimas: ", xb)
else:
    print("Solução atual não é ótima.")
    print("Valor atual da solução: ", valor_atual)
    print("Variáveis: ", xb)

Solução atual é ótima.
Valor ótimo da solução:  -35.14285714285714
Variáveis ótimas:  [ 4.28571429 11.14285714 -1.71428571]


In [132]:
# Cálculo da Direção Simplex
direcao_simplex = np.linalg.solve(particao_basica, particao_nbasica[:,[indice_entrada_base]])

if True in (direcao_simplex <= 0):
    print("Problema não tem solução ótima finita.")
# else:
    # epsilon =

Problema não tem solução ótima finita.
